### Машинное Обучения

## Домашнее задание по лекции - Деревья решений

**Общая информация**

**Срок сдачи:** 2 декабря 2024, 23:59   
**Штраф за опоздание:** -2 балла за каждые 2 дня опоздания

Решение залить в свой github репозиторий.

Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс MyDecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict . (Но название класса не менять)

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [34]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
        self.tree = dict()

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        # Ваш код здесь
        num_samples,num_features = x.shape
        best_gain = 0
        best_feature_id = None
        best_threshold = None

        if self.criterion == 'gini':
            impurity_func = self.__gini_impurity
        elif self.criterion == 'entropy':
            impurity_func = self.__entropy_impurity
        else:
            raise ValueError("Invalid criterion")

        current_impurity = impurity_func(y)

        for feature_id in range(num_features):
            unique_values = np.unique(x[:, feature_id])
            for value in unique_values:
                left_x, right_x, left_y, right_y = self.__div_samples(x, y, feature_id, value)
                if len(left_y) == 0 or len(right_y) == 0:
                    continue
                left_impurity = impurity_func(left_y)
                right_impurity = impurity_func(right_y)
                p_left = len(left_y) / num_samples
                p_right = len(right_y) / num_samples
                gain = current_impurity - (p_left * left_impurity + p_right * right_impurity)
                if gain > best_gain:
                    best_gain = gain
                    best_feature_id = feature_id
                    best_threshold = value

        return best_feature_id, best_threshold

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        # Ваш код здесь
        num_samples, num_features = x.shape

        if depth >= self.max_depth or num_samples < self.min_samples_split or len(np.unique(y)) == 1:
            class_prediction = np.bincount(y).argmax()
            class_probs = np.bincount(y) / len(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, class_prediction, class_probs)
            return

        best_feature_id, best_threshold = self.__find_threshold(x, y)

        if best_feature_id is None:
            class_prediction = np.bincount(y).argmax()
            class_probs = np.bincount(y) / len(y)
            self.tree[node_id] = (self.__class__.LEAF_TYPE, class_prediction, class_probs)
            return

        self.tree[node_id] = (self.__class__.NON_LEAF_TYPE, best_feature_id, best_threshold)

        left_x, right_x, left_y, right_y = self.__div_samples(x, y, best_feature_id, best_threshold)

        self.__fit_node(left_x, left_y, 2 * node_id + 1, depth + 1)
        self.__fit_node(right_x, right_y, 2 * node_id + 2, depth + 1)
    
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
    
    def get_feature_importance():
        """
        Возвращает важность признаков
        """
        # Ваш код здесь
        num_samples, num_features = self.tree[0][1].shape
        feature_importances = np.zeros(num_features)

        for node_id, node in self.tree.items():
            if node[0] == self.__class__.NON_LEAF_TYPE:
                feature_id, _ = node[1], node[2]
                feature_importances[feature_id] += 1

        feature_importances /= np.sum(feature_importances)
        self.feature_importances_ = feature_importances
        return self.feature_importances_

    def __gini_impurity(self, y):
        num_classes = len(np.unique(y))
        impurity = 1
        class_counts = np.bincount(y)
        for count in class_counts:
            p = count / len(y)
            impurity -= p ** 2
        return impurity

    def __entropy_impurity(self, y):
        num_classes = len(np.unique(y))
        impurity = 0
        class_counts = np.bincount(y)
        for count in class_counts:
            p = count / len(y)
            if p > 0:
                impurity -= p * np.log2(p)
        return impurity

In [36]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [38]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [40]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.8888888888888888

Совет: Проверьте, что ваша реализация корректно работает с признаками в которых встречаются повторы. 
И подумайте, какие еще граничные случаи могут быть.
Например, проверьте, что на таком примере ваша модель корректно работает:

In [43]:
X = np.array([[1] * 10, [0, 1, 2, 5, 6, 3, 4, 7, 8, 9]]).T
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
for depth in range(1, 5):
    my_clf = MyDecisionTreeClassifier(max_depth=depth)
    my_clf.fit(X, y)
    print("DEPTH:", depth, "\n\t\tTree:", my_clf.tree, my_clf.predict(X))

DEPTH: 1 
		Tree: {0: (0, 1, 2), 1: (1, 1, array([0.28571429, 0.71428571])), 2: (1, 0, array([1.]))} [0 0 0 1 1 1 1 1 1 1]
DEPTH: 2 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (1, 1, array([0., 1.])), 4: (1, 0, array([0.5, 0.5])), 2: (1, 0, array([1.]))} [0 0 0 0 0 0 0 1 1 1]
DEPTH: 3 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (1, 1, array([0., 1.])), 4: (0, 1, 4), 9: (1, 0, array([1.])), 10: (1, 1, array([0., 1.])), 2: (1, 0, array([1.]))} [0 0 0 0 0 1 1 1 1 1]
DEPTH: 4 
		Tree: {0: (0, 1, 2), 1: (0, 1, 6), 3: (1, 1, array([0., 1.])), 4: (0, 1, 4), 9: (1, 0, array([1.])), 10: (1, 1, array([0., 1.])), 2: (1, 0, array([1.]))} [0 0 0 0 0 1 1 1 1 1]


## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [46]:
%time clf.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 1 ms


DecisionTreeClassifier()

In [48]:
%time my_clf.fit(X_train, y_train)

CPU times: total: 31.2 ms
Wall time: 61.8 ms


## Боевое применение (3 балла)

Работаем с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Данные и описания колонок во вложениях.

Либо можно воспользоваться функцией:

In [52]:
def preprocess_spd_data(df):
    df = df.iloc[:, :97]
    
    to_drop = [
        'id', 'idg', 'condtn', 'round', 'position', 'positin1', 'order', 'partner', 
        'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
        'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o','met_o',
        'field', 'undergra', 'from', 'zipcode', 'income', 'career', 'sports', 'tvsports', 'exercise', 
        'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 
        'concerts', 'music', 'shopping', 'yoga', 'expnum',
        'mn_sat', 'tuition'
    ]

    df = df.drop(to_drop, axis=1)
    df = df.dropna(subset=['age', 'imprelig', 'imprace', 'date'])

    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
    
    # attr1 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 
                                            'amb1_1', 'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] =\
    (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / 
     df.loc[:, 'temp_totalsum'].T).T * 100
    
    # attr2 processing
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 
                                            'amb2_1', 'shar2_1']].sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] =\
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / 
     df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)
    
    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
                'intel{}_1'.format(i), 'fun{}_1'.format(i), 
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]

        if i != 4:
            feat.remove('shar{}_1'.format(i))
    
        df = df.drop(feat, axis=1)
    
    df = df.drop(['wave'], axis=1)
    df = df.dropna()
    return df

Скачайте датасет, обработайте данные, обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. В качестве метрики можно взять roc-auc. 


In [57]:
df = pd.read_csv("Speed_Dating_Data.csv", encoding='latin-1')
df = preprocess_spd_data(df)

X = df.drop('match', axis=1)
y = df['match']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

depth_values = [3, 5, 7, 9]
min_samples_split_values = [2, 5, 10]

train_accuracies = []
val_accuracies = []

In [59]:
train_roc_auc_scores = []
val_roc_auc_scores = []

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации и трейне от глубины дерева, от минимального числа объектов для сплита. (Т.е должно быть 2 графика, на каждой должны быть 2 кривые - для трейна и валидации)
Какой максимальной точности удалось достигнуть?

In [93]:
for depth in depth_values:
    for min_samples_split in min_samples_split_values:
        my_clf = MyDecisionTreeClassifier(max_depth=depth, min_samples_split=min_samples_split)

        my_clf.fit(X_train, y_train)

        y_train_pred = my_clf.predict(X_train)
        train_accuracy = (y_train_pred == y_train).mean()
        train_roc_auc = roc_auc_score(y_train, my_clf.predict_proba(X_train)[:, 1])

        y_val_pred = my_clf.predict(X_val)
        val_accuracy = (y_val_pred == y_val).mean()
        val_roc_auc = roc_auc_score(y_val, my_clf.predict_proba(X_val)[:, 1])

        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        train_roc_auc_scores.append(train_roc_auc)
        val_roc_auc_scores.append(val_roc_auc)

plt.figure(figsize=(10, 6))
for i, min_samples_split in enumerate(min_samples_split_values):
    plt.plot(depth_values, [train_accuracies[j] for j in range(len(depth_values)) if
                            ((j % len(min_samples_split_values)) == i)], label=f'min_samples_split={min_samples_split} (Train)')
    plt.plot(depth_values, [val_accuracies[j] for j in range(len(depth_values)) if
                            ((j % len(min_samples_split_values)) == i)], label=f'min_samples_split={min_samples_split} (Validation)')
plt.xlabel('Tree Depth')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Tree Depth')
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
for i, depth in enumerate(depth_values):
    plt.plot(min_samples_split_values, [train_accuracies[j] for j in range(len(min_samples_split_values)) if
                            ((j // len(min_samples_split_values)) == i)], label=f'depth={depth} (Train)')
    plt.plot(min_samples_split_values, [val_accuracies[j] for j in \
                            ((j // len(min_samples_split_values)) == i)], label=f'depth={depth} (Validation)')
    plt.xlabel('Minimum Samples to Split')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Minimum Samples to Split')
    plt.legend()
    plt.show()

max_accuracy = max(val_accuracies)
print(f"max_accuracy: {max_accuracy}")


InvalidIndexError: (slice(None, None, None), 0)

Известным фактом является то, что деревья решений сильно переобучаются при увеличении глубины и просто запоминают трейн. 
Замечаете ли вы такой эффект судя по графикам? Что при этом происходит с качеством на валидации? 

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ



In [ ]:
# It's too hard for me, I think I need more time